In [ ]:
from dateutil import rrule
from datetime import datetime
import requests
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [69]:
today = datetime.now()

BASE_URL = "https://api.weather.com/v1/location/VVNB:9:VN/observations/historical.json?apiKey=e1f10a1e78da46f5b10a1e78da96f525&units=e&startDate="


month = str(today.month)
day = str(today.day)
if len(month) == 1:
  month = '0' + month
if len(day) == 1:
  day = '0' + day
url = BASE_URL + str(d.year) + month + day

data = []

page = requests.get(url)
content = json.loads(page.content)
observation = content['observations']
for obs in observation:
  row = [obs['temp'], obs['rh'], obs['pressure'], obs['uv_index'], obs['wx_phrase'], obs['wdir'], obs['wspd']]
  data.append(row)

data[-1][-2]

170

In [64]:
datetime.now()

datetime.datetime(2021, 6, 2, 11, 3, 1, 73676)

In [59]:

datestr

'202162'

In [45]:
days_in_a_year = rrule.rrule(
        rrule.DAILY,
        dtstart=datetime.strptime('202162', '%Y%m%d'),
        until=datetime.strptime('202162', '%Y%m%d'))

In [46]:
BASE_URL = "https://api.weather.com/v1/location/VVNB:9:VN/observations/historical.json?apiKey=e1f10a1e78da46f5b10a1e78da96f525&units=e&startDate="
list_url = []

for d in days_in_a_year:
  month = str(d.month)
  day = str(d.day)
  if len(month) == 1:
    month = '0' + month
  if len(day) == 1:
    day = '0' + day
  list_url.append(BASE_URL + str(d.year) + month + day)

In [48]:
data = []
for url in list_url:
  page = requests.get(url)
  content = json.loads(page.content)
  try:
    observation = content['observations']
    for obs in observation:
      row = [obs['temp'], obs['rh'], obs['pressure'], obs['uv_index'], obs['wx_phrase'], obs['wdir'], obs['wspd']]
      data.append(row)
  except:
    pass

In [50]:
data[-1]

[99, 42, 29.46, 0, 'Fair', 170, 8]

In [ ]:
csv = 'Temperature,Humidity,Pressure,UV index,Conclusion,Wind direction,Wind speed'

for i, d in enumerate(data):
  csv += '\n' + ','.join([str(w) for w in d])

In [ ]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
import pandas as pd

TESTDATA = StringIO(csv)

data = pd.read_csv(TESTDATA, sep=",", index_col=False, encoding='unicode_escape')
# saving the dataframe
data.to_csv(r'/content/drive/MyDrive/Weather IoT/Noi Bai.csv', index=False, encoding="utf-8-sig")
data.head()

,Temperature,Humidity,Pressure,UV index,Conclusion,Wind direction,Wind speed
0,59,88,30.16,0,Fog,270,3
1,59,88,30.19,0,Fog,None,1
2,57,94,30.16,0,Fog,270,3
3,57,88,30.16,0,Fog,None,1
4,55,94,30.16,0,Fog,90,3


In [ ]:
import pandas as pd
import numpy as np

CSV_LIST = ['Cat Bi', 'Da Nang', 'Noi Bai', 'Phu Bai', 'Tan Son Nhat']
CSV_BASE = '/content/drive/MyDrive/Weather IoT/'

for csv in CSV_LIST:
  csv_path = CSV_BASE + csv + '.csv'

  df = pd.read_csv(csv_path)
  df = df.rename(columns={" Humidity": "Humidity", " Pressure": "Pressure", " UV index": "UV index", " Conclusion": "Conclusion"})

  for i, x in enumerate(df['Temperature']):
    df['Temperature'][i] = np.nan if x == 'None' else int(x)
  for i, x in enumerate(df['Humidity']):
    df['Humidity'][i] = np.nan if x == 'None' else int(x)
  for i, x in enumerate(df['Pressure']):
    df['Pressure'][i] = np.nan if x == 'None' else float(x)
  for i, x in enumerate(df['UV index']):
    df['UV index'][i] = np.nan if x == 'None' else int(x)

  df[["Temperature", "Humidity", "Pressure", "UV index"]] = df[["Temperature", "Humidity", "Pressure", "UV index"]].apply(pd.to_numeric)
  df.to_csv(csv_path, index=False, encoding="utf-8-sig")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyW

In [ ]:
import pandas as pd
import math

def mean(a):
    return sum(a)/len(a)

def fill_none(array_col):
    for i in range(len(array_col)):
      if math.isnan(array_col[i]):
        array_col[i] = mean([tmp for tmp in array_col[i-4:i+4] if math.isnan(tmp) == False])
    return array_col

CSV_LIST = ['Cat Bi', 'Da Nang', 'Noi Bai', 'Phu Bai', 'Tan Son Nhat']
CSV_BASE = '/content/drive/MyDrive/Weather IoT/'

for csv in CSV_LIST:
  csv_path = CSV_BASE + csv + '.csv'

  df = pd.read_csv(csv_path)

  df['Humidity']= fill_none(df['Humidity'].values)
  df['Pressure'] = fill_none(df['Pressure'].values)
  df['Temperature'] = fill_none(df['Temperature'].values)
  df['UV index'] = fill_none(df['UV index'].values)

  check_nan = df['Temperature'].isnull()
  print(check_nan.value_counts())
  check_nan = df['Pressure'].isnull()
  print(check_nan.value_counts())
  check_nan = df['UV index'].isnull()
  print(check_nan.value_counts())
  check_nan = df['Humidity'].isnull()
  print(check_nan.value_counts())

  df.to_csv(csv_path, index=False, encoding="utf-8-sig")

False    18274
Name: Temperature, dtype: int64
False    18274
Name: Pressure, dtype: int64
False    18274
Name: UV index, dtype: int64
False    18274
Name: Humidity, dtype: int64
False    109931
Name: Temperature, dtype: int64
False    109931
Name: Pressure, dtype: int64
False    109931
Name: UV index, dtype: int64
False    109931
Name: Humidity, dtype: int64
False    110306
Name: Temperature, dtype: int64
False    110306
Name: Pressure, dtype: int64
False    110306
Name: UV index, dtype: int64
False    110306
Name: Humidity, dtype: int64
False    104427
Name: Temperature, dtype: int64
False    104427
Name: Pressure, dtype: int64
False    104427
Name: UV index, dtype: int64
False    104427
Name: Humidity, dtype: int64
False    110420
Name: Temperature, dtype: int64
False    110420
Name: Pressure, dtype: int64
False    110420
Name: UV index, dtype: int64
False    110420
Name: Humidity, dtype: int64


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Weather IoT/Phu Bai.csv')
check_nan = df['Temperature'].isnull()
check_nan.value_counts()

False    104375
True         52
Name: Temperature, dtype: int64